<a href="https://colab.research.google.com/github/studorache/MLC/blob/master/MLC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade kaggle
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json


Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 63 bytes


In [0]:
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import pandas as pd
import os

Download and load train data into panda df

In [0]:
#download data
!kaggle competitions download -c ml-challenge-week6

#unzip data
!unzip census-income.data.zip
!unzip census-income.test.zip

census-income.test.zip: Skipping, found more recently modified local copy (use --force to force download)
census-income.data.zip: Skipping, found more recently modified local copy (use --force to force download)
census-income.names: Skipping, found more recently modified local copy (use --force to force download)
sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
Archive:  census-income.data.zip
replace census-income.data? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: census-income.data      
Archive:  census-income.test.zip
replace census-income.test? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: census-income.test      


In [0]:
#create panda dataframe from training data
import pandas as pd 

train_df = pd.read_csv('census-income.data', header=None,sep=',')
test_df = pd.read_csv('census-income.test', header=None,sep=',')

In [0]:
#get cols names
lines = open("census-income.names", "r").readlines()
i = 0
col_names = []
while i < len(lines):
    if i >= 81 and i<= 120:
      s = lines[i]
      col_names.append(s[s.find("(")+1:s.find(")")])
    i += 1
col_names.insert(col_names.index('detailed household summary in household')+1, 'instance weight') 
col_names

['age',
 'class of worker',
 'detailed industry recode',
 'detailed occupation recode',
 'education',
 'wage per hour',
 'enroll in edu inst last wk',
 'marital stat',
 'major industry code',
 'major occupation code',
 'race',
 'hispanic origin',
 'sex',
 'member of a labor union',
 'reason for unemployment',
 'full or part time employment stat',
 'capital gains',
 'capital losses',
 'dividends from stocks',
 'tax filer stat',
 'region of previous residence',
 'state of previous residence',
 'detailed household and family stat',
 'detailed household summary in household',
 'instance weight',
 'migration code-change in msa',
 'migration code-change in reg',
 'migration code-move within reg',
 'live in this house 1 year ago',
 'migration prev res in sunbelt',
 'num persons worked for employer',
 'family members under 18',
 'country of birth father',
 'country of birth mother',
 'country of birth self',
 'citizenship',
 'own business or self employed',
 "fill inc questionnaire for veteran

In [0]:
#Append cols list to test panda
test_df.columns = col_names

col_names.append('income class')
#Append cols list to train panda
train_df.columns = col_names

In [0]:
train_df.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,instance weight,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year,income class
0,73,Not in universe,0,0,High school graduate,0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,0,95,- 50000.
1,58,Self-employed-not incorporated,4,34,Some college but no degree,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,52,94,- 50000.
2,18,Not in universe,0,0,10th grade,0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0,0,0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0,Not in universe,2,0,95,- 50000.
3,9,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1758.14,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.
4,10,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,94,- 50000.


In [0]:
test_df.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,instance weight,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,year
0,38,Private,6,36,1st 2nd 3rd or 4th grade,0,Not in universe,Married-civilian spouse present,Manufacturing-durable goods,Machine operators assmblrs & inspctrs,White,Mexican (Mexicano),Female,Not in universe,Not in universe,Full-time schedules,0,0,0,Joint one under 65 & one 65+,Not in universe,Not in universe,Spouse of householder,Spouse of householder,1032.38,NaN,NaN,NaN,Not in universe under 1 year old,NaN,4,Not in universe,Mexico,Mexico,Mexico,Foreign born- Not a citizen of U S,0,Not in universe,2,12,95
1,44,Self-employed-not incorporated,37,12,Associates degree-occup /vocational,0,Not in universe,Married-civilian spouse present,Business and repair services,Professional specialty,White,All other,Female,Not in universe,Not in universe,PT for econ reasons usually PT,0,0,2500,Joint both under 65,Not in universe,Not in universe,Spouse of householder,Spouse of householder,1462.33,NaN,NaN,NaN,Not in universe under 1 year old,NaN,1,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,26,95
2,2,Not in universe,0,0,Children,0,Not in universe,Never married,Not in universe or children,Not in universe,White,Mexican-American,Male,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1601.75,NaN,NaN,NaN,Not in universe under 1 year old,NaN,0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,0,0,95
3,35,Private,29,3,High school graduate,0,Not in universe,Divorced,Transportation,Executive admin and managerial,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0,0,0,Head of household,Not in universe,Not in universe,Householder,Householder,1866.88,Nonmover,Nonmover,Nonmover,Yes,Not in universe,5,Not in universe,United-States,United-States,United-States,Native- Born in the United States,2,Not in universe,2,52,94
4,49,Private,4,34,High school graduate,0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Full-time schedules,0,0,0,Single,Not in universe,Not in universe,Secondary individual,Nonrelative of householder,1394.54,NaN,NaN,NaN,Not in universe under 1 year old,NaN,4,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0,Not in universe,2,50,95


In [0]:
categories_to_encode = ["class of worker", "education", "enroll in edu inst last wk", "marital stat", 
                       "major industry code", "major occupation code", "race", "hispanic origin", "sex", 
                       "member of a labor union", "reason for unemployment", "full or part time employment stat", 
                       "tax filer stat", "region of previous residence", "state of previous residence", 
                       "detailed household and family stat", "detailed household summary in household", 
                       "migration code-change in msa", "migration code-change in reg", 
                       "migration code-move within reg", "live in this house 1 year ago", 
                       "migration prev res in sunbelt", "family members under 18", "country of birth father", 
                       "country of birth mother", "country of birth self", "citizenship", 
                       "fill inc questionnaire for veteran's admin" ]

for cat in categories_to_encode:
  train_df[cat] = train_df[cat].astype('category')
  train_df[cat] = train_df[cat].cat.codes

train_df["income class"] = train_df["income class"].astype('category')
train_df["income class"] = train_df["income class"].cat.codes

for cat in categories_to_encode:
  test_df[cat] = test_df[cat].astype('category')
  test_df[cat] = test_df[cat].cat.codes


In [0]:
train_df.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,detailed household summary in household,instance weight,migration code-change in msa,migration code-change in reg,migration code-move within reg,live in this house 1 year ago,migration prev res in sunbelt,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,income class
0,73,3,0,0,12,0,2,6,14,6,4,0,0,1,3,2,0,0,0,4,3,36,24,6,1700.09,0,0,0,1,0,0,4,40,40,40,4,0,1,2,0,0
1,58,6,4,34,16,0,2,0,4,8,4,0,1,1,3,0,0,0,0,0,4,5,20,4,1053.55,3,8,9,0,3,1,4,40,40,40,4,0,1,2,52,0
2,18,3,0,0,0,0,1,4,14,6,1,0,0,1,3,2,0,0,0,4,3,36,2,0,991.95,0,0,0,1,0,0,4,41,41,41,0,0,1,2,0,0
3,9,3,0,0,10,0,2,4,14,6,4,0,0,1,3,0,0,0,0,4,3,36,8,2,1758.14,7,6,7,2,2,0,0,40,40,40,4,0,1,0,0,0
4,10,3,0,0,10,0,2,4,14,6,4,0,0,1,3,0,0,0,0,4,3,36,8,2,1069.16,7,6,7,2,2,0,0,40,40,40,4,0,1,0,0,0


In [0]:
test_df.head()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,instance weight,migration code-move within reg,live in this house 1 year ago,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year
0,38,4,6,36,3,0,2,2,10,5,4,6,0,1,3,1,0,0,0,3,3,36,36,1032.38,0,1,4,4,26,26,26,0,0,1,2,12
1,44,6,37,12,8,0,2,2,2,10,4,1,0,1,3,4,0,0,2500,2,3,36,36,1462.33,0,1,1,4,40,40,40,4,0,1,2,26
2,2,3,0,0,10,0,2,4,14,6,4,7,1,1,3,0,0,0,0,4,3,36,8,1601.75,0,1,0,0,40,40,40,4,0,1,0,0
3,35,4,29,3,12,0,2,0,21,2,4,1,0,1,3,0,0,0,0,0,3,36,19,1866.88,7,2,5,4,40,40,40,4,2,1,2,52
4,49,4,4,34,12,0,2,0,4,8,4,1,1,1,3,1,0,0,0,5,3,36,34,1394.54,0,1,4,4,40,40,40,4,0,1,2,50


In [0]:
# noise w/o correlation
train_df = train_df.drop("year", axis=1)
test_df = test_df.drop("year", axis=1)

In [0]:
# noise: hi corrr with detailed household and family stat
train_df = train_df.drop("detailed household summary in household", axis=1)
test_df = test_df.drop("detailed household summary in household", axis=1)

In [0]:
#noise: hi corr with migration code-move within reg
train_df = train_df.drop("migration code-change in reg", axis=1)
train_df = train_df.drop("migration prev res in sunbelt", axis=1)
train_df = train_df.drop("migration code-change in msa", axis=1)


test_df = test_df.drop("migration code-change in reg", axis=1)
test_df = test_df.drop("migration prev res in sunbelt", axis=1)
test_df = test_df.drop("migration code-change in msa", axis=1)

In [0]:
train_df.describe()

,age,class of worker,detailed industry recode,detailed occupation recode,education,wage per hour,enroll in edu inst last wk,marital stat,major industry code,major occupation code,race,hispanic origin,sex,member of a labor union,reason for unemployment,full or part time employment stat,capital gains,capital losses,dividends from stocks,tax filer stat,region of previous residence,state of previous residence,detailed household and family stat,instance weight,migration code-move within reg,live in this house 1 year ago,num persons worked for employer,family members under 18,country of birth father,country of birth mother,country of birth self,citizenship,own business or self employed,fill inc questionnaire for veteran's admin,veterans benefits,weeks worked in year,income class
count,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.00000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000,199523.000000
mean,34.494199,3.485378,15.352320,11.306556,10.036292,55.426908,1.908442,3.005573,13.006135,6.302075,3.644171,0.749608,0.478837,0.934825,3.009478,0.687339,434.71899,37.313788,197.529533,3.218576,3.007979,34.946307,20.059597,1740.380269,3.491537,1.334623,1.956180,3.053853,35.490124,35.666886,37.505686,3.618089,0.175438,0.993976,1.514833,23.174897,0.062058
std,22.310895,1.105278,18.067129,14.454204,4.120910,274.896454,0.374422,1.408378,4.778346,3.104561,0.862965,2.063948,0.499553,0.302160,0.315187,1.195851,4697.53128,271.896428,1984.163658,1.385113,0.463951,5.655596,11.426419,993.768156,3.593272,0.617057,2.365126,1.606296,10.590776,10.378807,8.179903,1.111627,0.553694,0.099536,0.851473,24.411488,0.241261
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,37.870000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,15.000000,3.000000,0.000000,0.000000,9.000000,0.000000,2.000000,2.000000,11.000000,6.000000,4.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.00000,0.000000,0.000000,2.000000,3.000000,36.000000,8.000000,1061.615000,0.000000,1.000000,0.000000,2.000000,40.000000,40.000000,40.000000,4.000000,0.000000,1.000000,2.000000,0.000000,0.000000
50%,33.000000,3.000000,0.000000,0.000000,10.000000,0.000000,2.000000,4.000000,14.000000,6.000000,4.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.00000,0.000000,0.000000,4.000000,3.000000,36.000000,20.000000,1618.310000,1.000000,1.000000,1.000000,4.000000,40.000000,40.000000,40.000000,4.000000,0.000000,1.000000,2.000000,8.000000,0.000000
75%,50.000000,4.000000,33.000000,26.000000,12.000000,0.000000,2.000000,4.000000,14.000000,7.000000,4.000000,0.000000,1.000000,1.000000,3.000000,1.000000,0.00000,0.000000,0.000000,4.000000,3.000000,36.000000,27.000000,2188.610000,7.000000,2.000000,4.000000,4.000000,40.000000,40.000000,40.000000,4.000000,0.000000,1.000000,2.000000,52.000000,0.000000
max,90.000000,8.000000,51.000000,46.000000,16.000000,9999.000000,2.000000,6.000000,23.000000,14.000000,4.000000,9.000000,1.000000,2.000000,5.000000,7.000000,99999.00000,4608.000000,99999.000000,5.000000,5.000000,50.000000,37.000000,18656.300000,9.000000,2.000000,6.000000,4.000000,42.000000,42.000000,42.000000,4.000000,2.000000,2.000000,2.000000,52.000000,1.000000


In [0]:
corr_matrix = train_df.corr()
corr_matrix.style.background_gradient()


In [0]:

X_train = train_df.drop(['income class'], axis=1)
y_train =train_df['income class']


In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Create the parameter grid
param_grid = {'max_depth': list(range(5,26)), 'max_features': ["auto", "sqrt", "log2", None]} 

# Create a random search object
random_rf_class = RandomizedSearchCV(
    estimator = RandomForestClassifier(n_estimators=80, random_state=42),
    param_distributions = param_grid, n_iter = 5,
    scoring='roc_auc', n_jobs=4, cv = 3, refit=True, return_train_score = True)



In [0]:
# Fit to the training data
random_rf_class.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix



/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
 random_rf_class.best_params_

{'criterion': 'gini',
 'max_depth': 12,
 'max_features': 'auto',
 'n_estimators': 100}

In [0]:
X_test = test_df
X_pred = random_rf_class.predict(X_test)

In [0]:
result = pd.DataFrame(X_pred)
#result.insert(0, 'index', result.index)
result.columns= ['income class']
result.to_csv('result.csv', index=True, index_label='index')

,income class
0,0
1,0
2,0
3,0
4,0


In [0]:
!kaggle competitions submit -c ml-challenge-week6 -f result.csv -m "First try."

100% 769k/769k [00:10<00:00, 73.1kB/s]
Successfully submitted to ML Challenge